In [1]:
import os
import tensorflow as tf
from PIL import Image
from nets import nets_factory
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from scipy.misc import imshow


In [2]:
CHAR_SET_LEN = 10
#image height
IMAGE_HEIGHT = 60
#image width
IMAGE_WIDTH = 160
#batch size
BATCH_SIZE = 1
#tfrecord file path
TFRECOD_FILE = "test.tfrecords"

#placeholder
x = tf.placeholder(tf.float32,[None,224,224])
# y0 = tf.placeholder(tf.float32,[None])
# y1 = tf.placeholder(tf.float32,[None])
# y2 = tf.placeholder(tf.float32,[None])
# y3 = tf.placeholder(tf.float32,[None])

#define the learning rate
lr = tf.Variable(0.003, dtype=tf.float32)


def read_and_decode(filename):
    filename_queue = tf.train.string_input_producer([filename])
    reader = tf.TFRecordReader()
    #The read function takes the filename queue, dequeues file names and extracts the image and label
    _,serialized_example = reader.read(filename_queue)
    #print len(serialized_example)
    print type(serialized_example)
    print "aaaaaa"
    features = tf.parse_single_example(serialized_example,
                                      features={
                                          'image':tf.FixedLenFeature([],tf.string),
                                          'label0':tf.FixedLenFeature([],tf.int64),
                                          'label1':tf.FixedLenFeature([],tf.int64),
                                          'label2':tf.FixedLenFeature([],tf.int64),
                                          'label3':tf.FixedLenFeature([],tf.int64),
                                      })
    # get the image data
    image = tf.decode_raw(features['image'],tf.uint8)
#     image_raw = image
    image = tf.reshape(image,[224,224])
    #image preprocessing
    image_raw = image
    image = tf.cast(image,tf.float32) / 255.0
    image = tf.subtract(image,0.5)
    image = tf.multiply(image,2.0)

    #get the label
    label0 = tf.cast(features['label0'],tf.int32)
    label1 = tf.cast(features['label1'],tf.int32)
    label2 = tf.cast(features['label2'],tf.int32)
    label3 = tf.cast(features['label3'],tf.int32)
    
    print(label0)
    return image,image_raw,label0,label1,label2,label3


In [3]:
# get the image and label infomation
image,image_raw,label0,label1,label2,label3 = read_and_decode(TFRECOD_FILE)


# # shuffle the input
image_batch,image_raw_batch,label_batch0,label_batch1,label_batch2,label_batch3 = tf.train.shuffle_batch(
    [image,image_raw,label0,label1,label2,label3],batch_size = BATCH_SIZE,
    capacity = 50000, min_after_dequeue = 10000,num_threads=1)
print type(image_batch)


# define the network structure
train_network_fn = nets_factory.get_network_fn('alexnet_v2',
                                              num_classes=CHAR_SET_LEN,
                                              weight_decay=0.0005,
                                              is_training= False)


X = tf.reshape(x,[BATCH_SIZE,224,224,1])
logits0,logits1,logits2,logits3,end_points = train_network_fn(X)

predict0 = tf.reshape(logits0,[-1, CHAR_SET_LEN]);
predict0 = tf.argmax(predict0,1);

predict1 = tf.reshape(logits1,[-1, CHAR_SET_LEN]);
predict1 = tf.argmax(predict1,1);

predict2 = tf.reshape(logits2,[-1, CHAR_SET_LEN]);
predict2 = tf.argmax(predict2,1);

predict3 = tf.reshape(logits3,[-1, CHAR_SET_LEN]);
predict3 = tf.argmax(predict3,1);



with tf.Session() as sess:
    
    print("check if it is correct")
    #initialzation
    sess.run(tf.global_variables_initializer())
    
    # load model
    saver = tf.train.Saver()
    saver.restore(sess,"captcha/models/crack_captcha.model-6000")
    print("model has been restored")
    # 
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    
    
    for i in range(10):
        b_image,b_image_raw,b_label10,b_label1,b_label2,b_label3 = sess.run([image_batch,image_raw_batch,
                                                                              label_batch0,
                                                                               label_batch1,
                                                                               label_batch2,
                                                                               label_batch3])
        label0,label1,label2,label3 = sess.run([predict0,predict1,predict2,predict3],feed_dict = {x : b_image})
        # show image
        img = Image.fromarray(b_image_raw[0],'L')

        fig,ax = plt.subplots(1)
        ax.imshow(img)
        s_label = "Predict:" + str(label0) + str(label1) + str(label2) + str(label3)
        s_label2 = "Label:" + str(b_label10) + str(b_label1) + str(b_label2) + str(b_label3)
        
        plt.text(x=10,y=10,s=str(s_label),bbox=dict(facecolor='white',alpha=1))
        plt.text(x=76,y=10,s=str(s_label2),bbox=dict(facecolor='white',alpha=1))
        plt.show()
        
        #print label
        print('label:',  b_label10,b_label1,b_label2,b_label3)
        
#         label0,label1,label2,label3 = sess.run([predict0,predict1,predict2,predict3],feed_dict = {x : b_image})
        #print predict
        print('predict:', label0,label1,label2,label3)
        
    coord.request_stop()
    coord.join(threads)
    

<class 'tensorflow.python.framework.ops.Tensor'>
aaaaaa
Tensor("Cast_1:0", shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.Tensor'>
check if it is correct
INFO:tensorflow:Restoring parameters from captcha/models/crack_captcha.model-6000
model has been restored
('label:', array([8], dtype=int32), array([5], dtype=int32), array([7], dtype=int32), array([9], dtype=int32))
('predict:', array([8]), array([5]), array([7]), array([9]))
('label:', array([4], dtype=int32), array([7], dtype=int32), array([3], dtype=int32), array([4], dtype=int32))
('predict:', array([4]), array([7]), array([3]), array([4]))
('label:', array([8], dtype=int32), array([1], dtype=int32), array([6], dtype=int32), array([0], dtype=int32))
('predict:', array([8]), array([1]), array([6]), array([0]))
('label:', array([5], dtype=int32), array([2], dtype=int32), array([1], dtype=int32), array([6], dtype=int32))
('predict:', array([5]), array([2]), array([1]), array([6]))
('label:', array([8], dtype=int32), 